# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver (colocar o executável na mesma pasta do Jupyter)

### Vamos fazer o passo a passo

- Passo 1: Pegar a cotação do Dólar
- Passo 2: Pegar a cotação do Euro
- Passo 3: Pegar a cotação do Ouro
- Passo 4: Importar a base de dados
- Passo 5: Atualizar a cotação, o Preço de Compra e o Preço de Venda
- Passo 6: Exportar o relatório atualizado

### Importar as bibliotecas

In [99]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

### Criar o navegador

In [100]:
# Informar o webdriver a ser usado
navegador = webdriver.Chrome('chromedriver.exe')

### Passo 1: Pegar a cotação do Dólar

In [101]:
# Entrar no Google
navegador.get('https://google.com.br')

# Pesquisar "cotação dólar"
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys('cotação dólar')

# Apertar "Enter"
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys(Keys.ENTER)

# Pegar a cotação do dólar
cotacao_dolar = navegador.find_element_by_xpath(
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
).get_attribute('data-value')
print(cotacao_dolar)

5.4939


### Passo 2: Pegar a cotação do Euro

In [102]:
# Entrar no Google
navegador.get('https://google.com.br')

# Pesquisar "cotação euro"
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys('cotação euro')

# Apertar "Enter"
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys(Keys.ENTER)

# Pegar a cotação do euro
cotacao_euro = navegador.find_element_by_xpath(
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
).get_attribute('data-value')
print(cotacao_euro)

6.350838522


### Passo 3: Pegar a cotação do Ouro

In [103]:
# Entrar no Melhor Câmbio
navegador.get('https://www.melhorcambio.com/ouro-hoje')

# Pegar a cotação do ouro
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')

# Tratar o valor pego (nesse caso, ele vem com vírgula)
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

311.66


### Passo 4: Importar a base de dados

In [104]:
# Armazenar o banco de dados em uma variável
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


### Passo 5: Atualizar a cotação, o Preço de Compra e o Preço de Venda

**Atualizar a cotação**

In [105]:
# Preencher as linhas de cotação do Dólar
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)  # primeiro a linha depois a coluna

# Preencher as linhas de cotação do Euro
tabela.loc[tabela['Moeda'] == 'Euro' ,'Cotação'] = float(cotacao_euro)  # primeiro a linha depois a coluna

# Preencher as linhas de cotação do Ouro
tabela.loc[tabela['Moeda'] == 'Ouro' ,'Cotação'] = float(cotacao_ouro)  # primeiro a linha depois a coluna

**Atualizar o Preço de Compra (Preço Original x Cotação)**

In [106]:
# Calcular o preço de compra
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

**Atualizar o Preço de Venda (Preço de Compra x Margem)**

In [107]:
# Calcular o preço final
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

In [108]:
# Formatar os valores
tabela['Preço de Compra'] = tabela['Preço de Compra'].map('{:.2f}'.format)
tabela['Preço de Venda'] = tabela['Preço de Venda'].map('{:.2f}'.format)

In [109]:
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.493900,5493.85,1.40,7691.38
1,Carro Renault,4500.00,Euro,6.350839,28578.77,2.00,57157.55
2,Notebook Dell,899.99,Dólar,5.493900,4944.46,1.70,8405.57
3,IPhone,799.00,Dólar,5.493900,4389.63,1.70,7462.36
4,Carro Fiat,3000.00,Euro,6.350839,19052.52,1.90,36199.78
5,Celular Xiaomi,480.48,Dólar,5.493900,2639.71,2.00,5279.42
6,Joia 20g,20.00,Ouro,311.660000,6233.20,1.15,7168.18


### Passo 6: Exportar o relatório atualizado

In [110]:
# Dentro do parênteses deve-se se passar o nome do documento final, não pode ser no original
tabela.to_excel('Produto Novo.xlsx', index=False)  # passar False para não gerar o arquivo com o index

# Fechar o navegador quando terminar tudo
navegador.quit()